# Extract information from a Wikipedia page and upload to Wikidata

This notebook takes an excerpt from a Wikipedia page about a scholar and extracts biographical information from it to upload the infromation to the WikiData enrty on that person. The steps are as follows:
 
1. send the excerpt to the OpenAi API (GPT-4), using a custom prompt that instructs the model to extract CSV data that can easily be arranged into statements and qualifiers
2. manually edit the data by correcting wrongly inferred information and adding missing triple data
3. upload the data using pywikibot  

## Definining the prompt

In [67]:
prompt = '''
Your task is to extract data from the text and to output it in a format that is suitable as a data source for adding triples to Wikidata.

The text is about "{fullName}" with the QID {qid}. It consists of one or more sections separated by "-----". The sections begin with a standalone URL followed by an excerpt of the content that can be found at this URL. 

Arrange the extracted information into a table with the following columns: subject-label, subject-qid, predicate, pid, object, object-qid, start_time, end_time, reference_url.  

Insert data into the columns as per the following rules:
- subject-label/subject-qid: In general, the subject is "{fullName}" with the QID {qid}. However, refining/qualifying statements can also be made about other entities, as with the academic degree (P512) item below. Also, in the case of P112, subject and object must be reversed
- predicate/pid:
    - educated at (P69): Institutions at which the person studied 
    - student of (P1066): If supervisors of doctoral theses and habilitations are specified
    - employer (P108): is the organization that pays the salary of a person (this can be a company, and institution or the university)
    - academic appointment (P8413): usually the department of a university, if this or its QID are not known, like P108
    - student (P802): persons contained in WikiData who were educated by the subject
    - member of (P463): Organizations and associations to which the person belongs (excluding P108)
    - affiliation (P1416): Organization that the subject is affiliated with (not member of or employed by)
    - academic degree (P512): some instance of academic degree (Q189533). After making this claim, add further triples to refine the P512 statement with triples on "conferred by" (P1027) and on "point in time" (P585).
    - editor (P98): add information on memberships in editorial boards of academic journals
    - founded by (P112): add information on journals, associations or other organizations that the subject helped to establish. When adding this claim, YOU MUST switch subject and object to express the reverse relationship
- object-label/object-qid: here the English labels and, if known, the QIDs for the institutions and persons who are the objects of the triple. If you are not absolutely sure, leave blank
- start_time: the date/year from which the triple statement is true. Leave blank if the date is not specified or cannot be inferred, or the triple involves P585 
- end_time: the date/year up to which the triple statement is true. If it is an event, identical to start_time
- reference_url: this is the source URL of the text from which the information was extracted.

Return information as a comma-separated values (CSV). Include the column headers. Surround the values with quotes. If values contain quotes, properly escape them.
 
DO NOT, UNDER ANY CIRCUMSTANCES, provide any commentary or explanations, just return the raw data. Do not make anything up that is not in the source material.
-----
{website_text}
'''


In [ ]:
## Data from Wikipedia (or any other website)

In [68]:
website_text = '''
https://de.wikipedia.org/wiki/Erhard_Blankenburg

Blankenburg belegte ein Studium der Philosophie, Soziologie und Germanistik an der Universität Freiburg und FU Berlin. Es folgten Graduate Studies und eine Tätigkeit als Forschungsassistent am Department of Sociology der University of Oregon. Ein Studium der Soziologie und Wirtschaftswissenschaft an der Universität Basel beendete er mit dem Abschluss Master of Arts 1965.

Seine Promotion zum Dr. phil. erfolgte an der Universität Basel 1966. Als Assistent am Institut für Soziologie der Universität Freiburg arbeitete er von 1966 bis 1968. Von 1969 bis 1971 war er Organisationsberater beim Quickborner Team, Hamburg. Danach arbeitete Blankenburg in Basel als Senior Projektleiter bei der Prognos in Basel. 1973/1974 war er wissenschaftlicher Mitarbeiter am Max-Planck-Institut für ausländisches und internationales Strafrecht in Freiburg. Die Habilitation für das Fach Soziologie erwarb er 1974 an der Universität Freiburg. Blankenburg war von 1975 bis 1980 Mitglied des Wissenschaftszentrums Berlin, Internationales Institut für Management und Verwaltung.

1980 bekam er einen Ruf auf den Lehrstuhl für Rechtssoziologie der Vrije Universiteit Amsterdam. Gemeinsam mit Wolfgang Kaupen spielte er eine wichtige Rolle bei der Neubegründung der Deutschen Rechtssoziologie in den 70er-Jahren (Raiser 1998), ebenso, mit Volkmar Gessner, bei der Gründung des International Institute for the Sociology of Law. Er gehörte auch zu den Initiatoren und zu den Gründungsherausgebern der Zeitschrift für Rechtssoziologie. Gemeinsam mit Bill Felstiner organisierte er 1991 in Amsterdam das erste gemeinsame Treffen der beiden bedeutenden Vereinigungen der Rechtssoziologie (LSA und RCSL). Seine Beschäftigung mit rechtssoziologischen Themen war ungewöhnlich breit, reichte von der Soziologie der Kriminalität über die des Staatsapparates bis zu der des Zivilrechts. Blankenburg war primär Empiriker und Methodiker (vgl. seine Empirische Rechtssoziologie). Seine wichtigsten Beiträge zur rechtssoziologischen Theorie betreffen die Begriffe der "Mobilisierung des Rechts" und der "Rechtskultur(en)". Vor allem aber wirkte er als Koordinator, Organisator und als Vermittler zwischen Wissenschaft und Praxis: "Er bemühte sich nicht, eine 'Schule' zu gründen, ihm fiel es leicht, in stets wechselnden Teams mit wechselnden Wissenschaftlern zusammenzuarbeiten. Wie kein anderer Rechtssoziologe vermochte er, erfolgreich Tagungen zu organisieren, kompetente Referenten zu gewinnen und die Veranstaltungen mit Autorität und zugleich locker zu leiten" (Theo Rasehorn 1998, 23). 
'''

## Query the OpenAI API (GPT-4)


In [69]:
import io
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

def use_model(model, template, debug=False, **params):
    prompt = ChatPromptTemplate.from_template(template)
    parser = StrOutputParser()
    chain = ( prompt | model | parser )
    response = chain.invoke(params)
    if debug:
        print(response)
    data = io.StringIO(response)
    return pd.read_csv(data, dtype={'start_time': str, 'end_time': str})

In [70]:
fullName = "Erhard Blankenburg"
qid="Q51595283"
model = ChatOpenAI(model_name="gpt-4")
df = use_model(model, prompt, fullName=fullName, qid=qid, website_text=website_text)
df.to_csv(f'data/{fullName}-chatgpt.csv', index=False)
df

,subject-label,subject-qid,predicate,pid,object,object-qid,start_time,end_time,reference_url
0,Erhard Blankenburg,Q51595283,educated at,P69,University of Freiburg,NaN,NaN,https://de.wikipedia.org/wiki/Erhard_Blankenburg,NaN
1,Erhard Blankenburg,Q51595283,educated at,P69,Free University of Berlin,NaN,NaN,https://de.wikipedia.org/wiki/Erhard_Blankenburg,NaN
2,Erhard Blankenburg,Q51595283,educated at,P69,University of Oregon,NaN,NaN,https://de.wikipedia.org/wiki/Erhard_Blankenburg,NaN
3,Erhard Blankenburg,Q51595283,educated at,P69,University of Basel,NaN,NaN,https://de.wikipedia.org/wiki/Erhard_Blankenburg,NaN
4,Erhard Blankenburg,Q51595283,academic degree,P512,Master of Arts,NaN,1965,1965,https://de.wikipedia.org/wiki/Erhard_Blankenburg
5,Master of Arts,NaN,conferred by,P1027,University of Basel,NaN,1965,1965,https://de.wikipedia.org/wiki/Erhard_Blankenburg
6,Erhard Blankenburg,Q51595283,employer,P108,University of Freiburg,NaN,1966,1968,https://de.wikipedia.org/wiki/Erhard_Blankenburg
7,Erhard Blankenburg,Q51595283,employer,P108,Quickborner Team,NaN,1969,1971,https://de.wikipedia.org/wiki/Erhard_Blankenburg
8,Erhard Blankenburg,Q51595283,employer,P108,Prognos,NaN,NaN,https://de.wikipedia.org/wiki/Erhard_Blankenburg,NaN
9,Erhard Blankenburg,Q51595283,employer,P108,Max-Planck-Institut für ausländisches und inte...,NaN,1973,1974,https://de.wikipedia.org/wiki/Erhard_Blankenburg


## Upload data to WikiData

The result can be seen at https://www.wikidata.org/wiki/Q51595283

In [73]:
# Code written by GPT-4
import csv
import pywikibot
from pywikibot import Claim, WbTime
from datetime import datetime

def claim_to_string(claim):
    """
    Converts a Pywikibot claim to a descriptive string in the format "(subject QID)-[predicate PID]-(object QID)".
    
    :param claim: A Pywikibot Claim object
    :return: A string describing the claim in the specified format.
    """
    # Subject QID, which is the ID of the item that owns the claim
    subject_qid = claim.on_item.id

    # Predicate PID, which is the property ID of the claim
    predicate_pid = claim.getID()

    # Object QID, assuming the target is a Wikidata item
    # Note: This simplification assumes the claim's target is an item. 
    # For other target types (e.g., quantities, strings), additional handling is needed.
    if isinstance(claim.getTarget(), pywikibot.ItemPage):
        object_qid = claim.getTarget().id
    else:
        # Placeholder or additional logic for non-ItemPage targets
        object_qid = 'N/A'  # This could be expanded to handle other types of targets

    return f"({subject_qid})-[{predicate_pid}]-({object_qid})"


def update_wikidata(file_path):
    site = pywikibot.Site("wikidata", "wikidata")
    repo = site.data_repository()

    previous_object_qid = None
    previous_claim = None

    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            subject_qid = row['subject-qid']
            pid = row['pid']
            object_qid = row['object-qid']
            start_time = row['start_time']
            end_time = row['end_time']
            reference_url = row['reference_url']

            # If the new subject is identical to the old object, refine the previous claim
            if subject_qid == previous_object_qid and previous_claim:
                claim = previous_claim
                print(f'Refining {claim_to_string(claim)}')
            else:
                item = pywikibot.ItemPage(repo, subject_qid)
                item.get()

                # Check if the claim already exists
                claim_exists = False
                for claim in item.claims.get(pid, []):
                    if claim.getTarget().id == object_qid:
                        claim_exists = True
                        print(f'{claim_to_string(claim)} exists.')
                        break

                if not claim_exists:
                    claim = Claim(repo, pid)
                    target = pywikibot.ItemPage(repo, object_qid)
                    claim.setTarget(target)
                    item.addClaim(claim)
                    print(f'Created {claim_to_string(claim)}')

            # Add qualifiers
            if start_time:
                start_time_qualifier = Claim(repo, 'P580')
                start_time_qualifier.setTarget(WbTime(year=int(start_time)))
                claim.addQualifier(start_time_qualifier, summary='Adding start time')
                print(f'Added start time')

            if end_time:
                end_time_qualifier = Claim(repo, 'P582')
                end_time_qualifier.setTarget(WbTime(year=int(end_time)))
                claim.addQualifier(end_time_qualifier, summary='Adding end time')
                print(f'Added end time')

            # Add reference
            if reference_url:
                source_claim = Claim(repo, 'P4656' if 'wikipedia' in reference_url else 'P854')
                source_claim.setTarget(reference_url)
                # add retrieved at
                retrieved_at_claim = Claim(repo, 'P813')
                retrieved_at_claim.setTarget(WbTime(year=datetime.utcnow().year, month=datetime.utcnow().month, day=datetime.utcnow().day))
                source_claim.addQualifier(retrieved_at_claim)
                claim.addSources([source_claim], summary='Adding reference')
                print(f"Added reference {reference_url} with access date")

            # Remember the object and claim for the next iteration
            previous_object_qid = object_qid
            previous_claim = claim

update_wikidata('data/Erhard Blankenburg.csv')

Created (Q51595283)-[P69]-(Q153987)


Sleeping for 9.5 seconds, 2024-03-15 18:19:54


Added reference https://de.wikipedia.org/wiki/Erhard_Blankenburg with access date


Sleeping for 9.2 seconds, 2024-03-15 18:20:04


Created (Q51595283)-[P69]-(Q153006)


...

Added reference https://de.wikipedia.org/wiki/Erhard_Blankenburg with access date
Refining (Q65972149)-[P112]-(Q51595283)


Sleeping for 9.5 seconds, 2024-03-15 18:27:54


Added end time


Sleeping for 9.6 seconds, 2024-03-15 18:28:04


Added reference https://www.linkedin.com/in/erhard-blankenburg-63938058/ with access date
